In [1]:
import numpy as np
import sys
from functools import partial
from ipycanvas import hold_canvas, Canvas
from tqdm import tqdm

sys.path.append('../src')
from models import Vicsek
from util import draw, grab_plot, plot, RunTime, VideoWriter

In [7]:
CANVAS_SIZE = 500
FARM_COLOR = '#1b1f22'
ANT_COLOR = 'red'
ANT_SIZE = 3.

def update():
    global model, canvas, x, v
    
    x, v = model(x, v)
    draw(x, canvas, CANVAS_SIZE, FARM_COLOR, ANT_COLOR, ANT_SIZE)

def toggle(*_):
    global runtime

    if runtime.running:
        runtime.stop()
    else:
        runtime.start()

def start(function, interval=.02):
    global runtime

    try:
        runtime.stop()
    except:
        pass

    runtime = RunTime(function, interval)

In [8]:
N_ANTS = 100
PROXIMITY = 20.
VELOCITY = 2.5
UPDATE_RATE = 1.
VARIANCE = .3

x = np.random.rand(N_ANTS, 2)*CANVAS_SIZE
v = 2*np.pi*np.random.rand(N_ANTS, 1)
model = Vicsek(PROXIMITY, VELOCITY, UPDATE_RATE, VARIANCE, CANVAS_SIZE)
canvas = Canvas(width=CANVAS_SIZE, height=CANVAS_SIZE)
canvas.on_mouse_down(toggle)

display(canvas)
draw(x, canvas, CANVAS_SIZE, FARM_COLOR, ANT_COLOR, ANT_SIZE)
start(update)

Canvas(width=500)